# Imports


In [33]:
import random
from operator import itemgetter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import data

In [3]:
raw_data6 = pd.read_csv('data/scenario6.labeled', delimiter='\s+')
# raw_data10 = pd.read_csv('data/scenario10.labeled', delimiter='\s+')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def parse_data(raw):
    data = raw
    data['Date'] = data['Date'].map(str) + " " + data['flow']

    src_ip_port = data['Prot'].map(str).str.split(':', n=1, expand = True)
    dst_ip_port = data['IP'].map(str).str.split(':', n=1, expand = True)
    data['src_ip'] = src_ip_port[0]
    data['src_port'] = src_ip_port[1]
    data['dst_ip'] = dst_ip_port[0]
    data['dst_port'] = dst_ip_port[1]

    data.drop(['flow', 'Src','Packets', 'Bytes', 'Flows', 'Prot', 'IP', 'Label', 'Labels'], axis=1, inplace=True)
    data.rename(columns={'start':'duration', 'Date':'start','Durat':'protocol', 'Addr:Port':'flags', 'Dst':'tos', 'IP.1':'packets', 'Addr:Port.1':'bytes', 'Flags':'flows', 'Tos':'label'}, inplace=True)
    data.sort_values(by='start', inplace=True)
    
    return data

In [5]:
data6 = parse_data(raw_data6)
print(data6.head())
print(data6['label'].unique())

                     start  duration protocol flags  tos  packets    bytes  \
0  2011-08-16 10:01:46.972     4.933      TCP    A_    0      537    37374   
1  2011-08-16 10:01:46.974     4.929      TCP   PA_    0      936  1417104   
2  2011-08-16 10:01:46.974     4.999      TCP   PA_    0      949   208122   
3  2011-08-16 10:01:46.976     0.000      UDP   INT    0        1      174   
4  2011-08-16 10:01:46.977     0.009      TCP   RA_    0        4      264   

   flows       label           src_ip src_port         dst_ip dst_port  
0      1  Background    88.176.79.163    49213  147.32.84.172    18250  
1      1  Background    147.32.84.172    18250  88.176.79.163    49213  
2      1  Background    147.32.85.112       22   85.70.14.207    10005  
3      1  Background      90.178.10.8    61997  147.32.84.229    13363  
4      1  Background  117.211.100.130    43458   147.32.86.92       80  
['Background' 'LEGITIMATE' 'Botnet']


# Sampling

### Obtain IP addresses to sample

In [131]:
df = data6
infected_ip = '147.32.84.165'

src_rows_with_infected_ip = df[df['src_ip'] == infected_ip]
dst_df = src_rows_with_infected_ip['dst_ip']
from_infected = dst_df.values

dst_rows_with_infected_ip = df[df['dst_ip'] == infected_ip]
src_df = dst_rows_with_infected_ip['src_ip']
to_infected = src_df.values

nr_of_unique_ips = len(np.unique(from_infected))

print(f'IP {infected_ip} sends packets to {nr_of_unique_ips} unique IP addresses.')    

IP 147.32.84.165 sends packets to 1582 unique IP addresses.


### Reservoir Sampling


Estimate the distribution over the other IP_addresses, what are the 10 most frequent values?
Write code for RESERVOIR sampling, use it to estimate the distribution in one pass (no need to
actually stream the data, you may store it in memory, or run every file separately, but do store
and load the intermediate results). Use a range of reservoir sizes. What are the 10 most frequent
IP-addresses and their frequencies when sampled? Use the theory to explain any approximation
errors you observe

Implemented correctly, explanation accurate

In [43]:
# random_replace replaces a random element in sample with ip
def random_replace(sample, ip):
    r = random.randint(0, len(sample) - 1)
    sample[r] = ip
    

# reservoir_sample performs reservoir sampling on the stream
def reservoir_sample(stream, reservoir_size):
    sample = []
    for i, ip in enumerate(stream):
        if i < reservoir_size:
            # Fill sample first
            sample.append(ip)
        else:
            # Probability that the next item should be sampled
            proba = reservoir_size / (i + 1)
            # Draw random number, if smaller than proba it should be sampled
            r = random.random()
            if r <= proba:
                # Element should be sampled
                random_replace(sample, ip)
    return sample


# aggregate counts the occurences per IP
def aggregate(ips, return_n_best=None):
    counts = {}
    for ip in ips:
        if ip not in counts:
            counts.update({ip: 0})
        counts[ip] += 1
    if return_n_best is None:
        return counts
    else:
        counts = counts.items()
        return sorted(counts, key=itemgetter(1), reverse=True)[:return_n_best]
    

# overlap computes the number of elements which are estimated 'correctly'
def overlap(real, estimate):
    real = set([ip for (ip, _) in real])
    estimate = [ip for (ip, _) in estimate]
    count = 0
    for e in estimate:
        if e in real:
            count +=1            
    return count

In [129]:
best_n = 10

# Real distribution
real = aggregate(from_infected, best_n)
# print(f'Reality {best_n} most frequent: {real}')

reservoir_size = 300
sample = reservoir_sample(from_infected, reservoir_size)
estimated = aggregate(sample, best_n)
# print(f'Estimated {best_n} most frequent: {estimated}')

ip_overlap = overlap(real, estimated)
print(f'Overlap: {ip_overlap}')

Overlap: 6


# Sketching

Build code for computing a COUNT-MIN sketch, play with different heights and widths for the Count-Min sketch matrix. Compare it to the RESERVOIR sampling strategy. Is it more spaceefficient/accurate? What about run-time? Use the theory to explain any differences you observe.

Implemented correctly, analysis, and evaluation are sound

### COUNT-MIN Sketch

In [73]:
_memomask = {}

# This function returns a (semi)unique hash function. 
# A new hash function is created based on the built-in hash function,
# but with a seeded random mask. Based on a stackoverflow code snippet.
# source: https://stackoverflow.com/questions/2255604/hash-functions-family-generator-in-python
def hash_function(n, w):
    mask = _memomask.get(n)
    if mask is None:
        random.seed(n)
        mask = _memomask[n] = random.getrandbits(64)
        
    def myhash(x):
        return (hash(x) ^ mask) % w
    
    return myhash

# Create a matrix with counts
def count_min_sketch(stream, w, d):
    cm = np.zeros((d, w), dtype=int)
    hash_functions = [hash_function(i, w) for i in range(d)]
    for e in stream:
        for i, hi in enumerate(hash_functions):
            v = hi(e)
            cm[i, v] += 1
    return cm, hash_functions

# Estimate occurence of a list of ips
def estimate_occurences(cm, hash_functions, ips):
    res = []
    for ip in ips:
        min_occurence = None
        for i, hi in enumerate(hash_functions):
            v = cm[i, hi(ip)]
            if min_occurence is None or min_occurence > v:
                min_occurence = v
        res.append((ip, min_occurence))
    return res

# w = 2 / epsilon
# epsilon = 2 / w
# d = log(1/delta)
# Map hash function to [1...w]
# resulting count is overestimate

In [85]:
stream = from_infected
# width
w = 50
# n_hash_functions, depth/height
d = 5
cm, hs = count_min_sketch(stream, w, d)
ips = [ip for (ip, occ) in real]
estimates = estimate_occurences(cm, hs, ips)
for i, (ip, occ) in enumerate(real):
    print(f'IP: {ip}, real occurences: {occ}, predicted occurences: {estimates[i][1]}')

IP: 91.212.135.158, real occurences: 384, predicted occurences: 437
IP: 64.59.134.8, real occurences: 194, predicted occurences: 231
IP: 24.71.223.11, real occurences: 160, predicted occurences: 217
IP: 216.32.180.22, real occurences: 154, predicted occurences: 211
IP: 65.55.37.72, real occurences: 96, predicted occurences: 130
IP: 65.55.37.88, real occurences: 94, predicted occurences: 133
IP: 65.55.92.136, real occurences: 93, predicted occurences: 145
IP: 65.55.92.168, real occurences: 90, predicted occurences: 123
IP: 65.55.92.152, real occurences: 90, predicted occurences: 146
IP: 202.108.252.141, real occurences: 89, predicted occurences: 121


In [133]:
print(len(from_infected))

5724
